In [0]:
!date

# Download and install kallisto|bustools




In [0]:
!wget https://github.com/pachterlab/kallisto/releases/download/v0.46.0/kallisto_linux-v0.46.0.tar.gz
!tar -xf kallisto_linux-v0.46.0.tar.gz
!cp kallisto/kallisto /usr/local/bin/

!wget https://github.com/BUStools/bustools/releases/download/v0.39.3/bustools_linux-v0.39.3.tar.gz
!tar -xf bustools_linux-v0.39.3.tar.gz
!cp bustools/bustools /usr/local/bin/

In [0]:
#TEST
!kallisto
!bustools

# Get reference data

In [0]:
# DOWLOAD kallisto index
!wget https://github.com/BUStools/getting_started/releases/download/getting_started/Mus_musculus.GRCm38.cdna.all.idx.gz
!gunzip Mus_musculus.GRCm38.cdna.all.idx.gz

# get the whitelist and tx to gene file
!wget https://github.com/BUStools/getting_started/releases/download/getting_started/10xv2_whitelist.txt
!wget https://github.com/BUStools/getting_started/releases/download/getting_started/transcripts_to_genes.txt

# Run kallisto (streaming fastqs as input)

In [0]:
!\
urlR1="https://github.com/bustools/getting_started/releases/download/getting_started/SRR8599150_S1_L001_R1_001.fastq.gz"; \
urlR2="https://github.com/bustools/getting_started/releases/download/getting_started/SRR8599150_S1_L001_R2_001.fastq.gz"; \
time kallisto bus -i Mus_musculus.GRCm38.cdna.all.idx -x 10xv2 -t 2 -o bus_out/ <(curl -Ls ${urlR1}) <(curl -Ls ${urlR2})

#Run bustools (correct, sort, count)

In [0]:
!mkdir bus_out/genecount/ bus_out/tmp/
!time bustools correct -w 10xv2_whitelist.txt -p bus_out/output.bus | bustools sort -T bus_out/tmp/ -t 2 -p - | bustools count -o bus_out/genecount/genes -g transcripts_to_genes.txt -e bus_out/matrix.ec -t bus_out/transcripts.txt --genecounts -

---



---

In [0]:
!date

---



---

# Install python packages




In [0]:
!pip install scanpy[louvain] 
!pip install MulticoreTSNE

# Getting Started Analysis: Analyzing SRR8599150 with ScanPy 

*   Jupyter notebook from [kallistobus.tools/getting_started](https://www.kallistobus.tools/getting_started) 

### Import Packages

In [0]:
import scanpy as sc
from scipy import sparse, io
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import numpy as np

matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

### Import MTX, GENES, BARCODES

In [0]:
folder = "/content/bus_out/genecount/"

The annotated dataframe has rows (obs) which are cell barcodes, columns (var) which are genes. The actual matrix `adata.X` is a sparse matrix.

In [0]:
adata = sc.read_mtx(folder + "genes.mtx")
adata.obs.index = pd.read_csv(folder + "genes.barcodes.txt", header=None)[0].values
adata.var.index = pd.read_csv(folder + "genes.genes.txt", header=None)[0].values

In [0]:
adata

## Make the knee plot

In [0]:
knee = np.sort((np.array(adata.X.sum(axis=1))).flatten())[::-1]

In [0]:
fig, ax = plt.subplots(figsize=(10, 7))

expected_num_cells=3949

ax.loglog(knee, range(len(knee)), label="kallisto", linewidth=5, color="k")
ax.axvline(x=knee[expected_num_cells], linewidth=3, color="g")
ax.axhline(y=expected_num_cells, linewidth=3, color="g")

ax.set_xlabel("UMI Counts")
ax.set_ylabel("Set of Barcodes")

plt.grid(True, which="both")
ax.legend()
plt.show()

## Analysis with ScanPy

In [0]:
sc.pp.filter_cells(adata, min_genes=0)
sc.pp.filter_cells(adata, min_counts=knee[expected_num_cells])
sc.pp.filter_genes(adata, min_cells=0)

In [0]:
adata

### Distribution of genes

In [0]:
fig, ax = plt.subplots(figsize=(10, 7))
sc.pl.violin(adata, 'n_genes', jitter=0.4, ax=ax)

### Distribution of counts

In [0]:
fig, ax = plt.subplots(figsize=(10, 7))
sc.pl.violin(adata, 'n_counts', jitter=0.4, ax=ax)

Normalize the counts in the matrix

In [0]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)

apply $log(1+count)$ to all counts in your matrix

In [0]:
adata.raw = sc.pp.log1p(adata, copy=True)

In [0]:
sc.pp.log1p(adata)

Filter adata for highly variable genes, and make `fadata` which only contains highly variable genes for further analysis.

### Highly variable genes

In [0]:
filter_result = sc.pp.highly_variable_genes(adata, min_disp=0.3,inplace=False, n_top_genes=1209)

sc.pl.highly_variable_genes(filter_result)

In [0]:
print("{:,} highly variable genes".format(sum(x[0] for x in filter_result)))

### Cluster with Louvain and Plot PCA, UMAP, TSNE

In [0]:
%%time
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.louvain(adata)

In [0]:
%%time
sc.tl.tsne(adata, n_pcs=10)

In [0]:
fig, ax = plt.subplots(figsize=(10, 7))
sc.pl.pca(adata, color="louvain", ax=ax)

In [0]:
sc.pl.pca_variance_ratio(adata)

In [0]:
fig, ax = plt.subplots(figsize=(10, 7))
sc.pl.tsne(adata, color="louvain", ax=ax, save="getting_started_tsne.png")

In [0]:
!date